# Topic Modeling

### Load relevant libraries and data

In [ ]:
# for data manipulation
import numpy as np
import pandas as pd
import ast

# for NLP
import nltk
from nltk.corpus import stopwords
import gensim
import gensim.corpora as corpora
from gensim import models
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import TfidfModel
import spacy
import pyLDAvis
import pyLDAvis.gensim

# for counter
from tqdm import tqdm

In [ ]:
# load the already pre-processed dataframe
df = pd.read_csv('df_tokenized.csv')

In [ ]:
df.head()

,tokens,sentiment
0,"['great', 'locat', 'close', 'main', 'public', ...",positive
1,"['famili', 'four', 'thi', 'flat', 'can', 'acco...",positive
2,"['place', 'wonder', 'plenti', 'room', 'us', 'h...",positive
3,"['great', 'locat', 'truli', 'onli', 'coupl', '...",positive
4,"['great', 'place', 'perfect', 'weekend', 'not'...",positive


In following steps only nouns, adjectives, verbs and adverbs are taken into account!

In [ ]:
# convert to list of lists

# get back tokenized form
data = df['tokens'].apply(ast.literal_eval)

# convert back to a list of lists
data = data.tolist()

In [ ]:
# initiate spacy pos tagging
nlp = spacy.load('en_core_web_sm')

In [ ]:
# only including nouns, adjectives, verbs and adverbs
final_data = []

for tokens in tqdm(data):
    review = nlp(' '.join(tokens))
    new_rev = []
    for word in review:
        if word.pos_ in ['NOUN', 'ADJ', 'VERB', 'ADV']:
            new_rev.append(word)
    final_data.append(new_rev)

100%|██████████| 660645/660645 [2:12:26<00:00, 83.14it/s]   


In [ ]:
# convert spacy format back to string
data = [[str(token) for token in tokens] for tokens in tqdm(final_data)]

100%|██████████| 660645/660645 [00:14<00:00, 46297.37it/s]


In [ ]:
# convert to dataframe
df = pd.DataFrame(
    {'tokens': data,
     'sentiment': df['sentiment']
    })

In [ ]:
# save df as csv
# df.to_csv('df_LDA.csv', index=False)

In [ ]:
# load the data
df = pd.read_csv('topic_modeling_final.csv')

In [ ]:
# convert to list of lists

# get back tokenized form
data = df['tokens'].apply(ast.literal_eval)

# convert back to a list of lists
data = data.tolist()

In [ ]:
data[:2]

[['great',
  'close',
  'main',
  'public',
  'transport',
  'easi',
  'anywher',
  'get',
  'super',
  'easi',
  'check',
  'veri',
  'respons',
  'clean',
  'close_main',
  'main_public',
  'public_transport',
  'easi_get',
  'get_anywher',
  'get_back',
  'super_easi',
  'check_veri',
  'veri_respons',
  'respons_clean',
  'clean_well',
  'stock_thank',
  'transport_easi_get',
  'clean_well_stock'],
 ['famili',
  'thi',
  'flat',
  'easili',
  'lot',
  'space',
  'edeka',
  'corner',
  'get',
  'need',
  'great',
  'stay',
  'four_thi',
  'thi_flat',
  'flat_can',
  'can_accommod',
  'accommod_person',
  'person_easili',
  'easili_lot',
  'lot_space',
  'space_us',
  'can_get',
  'get_everyth',
  'need_great',
  'great_stay',
  'thi_flat_can',
  'around_corner_can',
  'corner_can_get',
  'can_get_everyth',
  'everyth_need_great']]

### Data pre-processing

Tokenization, lemmatization, stemming have already been done. The following code does the other steps necessary for topic modeling.

##### Create dictionary with all unique words: id2word

In [ ]:
# create id2word which contains all unique words used in all reviews
id2word = corpora.Dictionary(data)

In [ ]:
# create a list containing all reviews with their respective used words and word count
corpus = []
for text in data:
    new = id2word.doc2bow(text)
    corpus.append(new)

In [ ]:
# check the first review
print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1)]


However, we will filter for extreme values later to get a better result and create a new list that will replace this one.

In [ ]:
# check that length of corpus is as long as the amount of documents
len(corpus)

660645

In [ ]:
# remove duplicate documents
corpus = [x for x in corpus if x]

In [ ]:
# check new length of corpus
len(corpus)

659339

##### Filter for extreme values

Tokens that appear in less than 30 reviews are filtered out or in more than 80% of reviews will be filtered out.

Finally after both first steps, we keep the 10,000 most frequent tokens.

We played with the thresholds / parameters to obtain a satisfying result.

In [ ]:
# filter for extreme values
id2word.filter_extremes(no_below=30, no_above=0.8, keep_n=10000)

In [ ]:
# check new corpus length
print(len(id2word.iteritems()))

10000


##### Create a list containing all reviews with their respective used words and word count: bow_corpus

In [ ]:
bow_corpus = [id2word.doc2bow(doc) for doc in data]

In [ ]:
# check the first review
print(bow_corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1)]


In [ ]:
# check that length of corpus is as long as the amount of documents
print(len(bow_corpus))

660645


In [ ]:
# check the representation with actual words
first_review = bow_corpus[0]

for token in range(len(first_review)):
    print('Word {} (\"{}\") appears {} time.'.format(first_review[token][0], id2word[first_review[token][0]], first_review[token][1]))

Word 0 ("anywher") appears 1 time.
Word 1 ("check") appears 1 time.
Word 2 ("check_veri") appears 1 time.
Word 3 ("clean") appears 1 time.
Word 4 ("clean_well") appears 1 time.
Word 5 ("clean_well_stock") appears 1 time.
Word 6 ("close") appears 1 time.
Word 7 ("close_main") appears 1 time.
Word 8 ("easi") appears 2 time.
Word 9 ("easi_get") appears 1 time.
Word 10 ("get") appears 1 time.
Word 11 ("get_anywher") appears 1 time.
Word 12 ("get_back") appears 1 time.
Word 13 ("great") appears 1 time.
Word 14 ("main") appears 1 time.
Word 15 ("public") appears 1 time.
Word 16 ("public_transport") appears 1 time.
Word 17 ("respons") appears 1 time.
Word 18 ("super") appears 1 time.
Word 19 ("super_easi") appears 1 time.
Word 20 ("transport") appears 1 time.
Word 21 ("veri") appears 1 time.
Word 22 ("veri_respons") appears 1 time.


### Apply tf-idf scores to the corpus

In [ ]:
# apply tf-idf to the bow_corpus
tfidf_model = TfidfModel(bow_corpus)
corpus_tfidf = tfidf_model[bow_corpus]

### Running LDA using Bag-of-Words

Latent Dirichlet Allocation utilizes the tf-idf scores. 

The tf-idf scores rank the importance of a token in a review given a corpus of reviews.

The idea is that the more a token appears in a document, the higher the token is ranked (tf: term frequency).
However, the term gets a lower weight the more it appears in other documents (idf: inverse document frequency).

We played with the parameters to obtain a satisfying result (e.g., number of topics etc.).

In [ ]:
# initiate model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus_tfidf, 
                                            id2word=id2word,
                                            num_topics=6,
                                            random_state=42,
                                            passes=5,
                                            alpha='auto')

### Visualizing topics

In [ ]:
# for visualization
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, id2word, mds='mmds', R=15)

/opt/conda/lib/python3.10/site-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


In [ ]:
# visualize
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.109963 -0.159774       1        1  64.486093
0     -0.228624  0.342100       2        1  11.488434
5      0.151054 -0.421006       3        1   9.538560
1      0.212131  0.367821       4        1   6.396793
2      0.426395 -0.018016       5        1   5.929485
4     -0.450993 -0.111125       6        1   2.160634, topic_info=              Term           Freq          Total Category  logprob  loglift
13           great  113788.000000  113788.000000  Default  15.0000  15.0000
46           place   95520.000000   95520.000000  Default  14.0000  14.0000
120           good   62983.000000   62983.000000  Default  13.0000  13.0000
84     great_place   30777.000000   30777.000000  Default  12.0000  12.0000
510     place_stay   30535.000000   30535.000000  Default  11.0000  11.0000
...            ...            ...            ...      ...      ...      ...
2555  host_perfect    2386.032726    2390.893895   Topic6  -4.9510   3.8327
3191     reccomend    2196.105600    2200.967491   Topic6  -5.0340   3.8326
2557     kind_host    1977.879077    1982.744461   Topic6  -5.1386   3.8323
3248        sunset    1944.067651    1948.946517   Topic6  -5.1559   3.8323
9954          oslo    2199.302213   25732.097641   Topic6  -5.0325   1.3752

[165 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
92        1  0.999745  access
92        2  0.000052  access
92        3  0.000052  access
92        4  0.000052  access
92        5  0.000052  access
...     ...       ...     ...
60        2  0.001713  wonder
60        3  0.000023  wonder
60        4  0.000023  wonder
60        5  0.000023  wonder
60        6  0.000023  wonder

[888 rows x 3 columns], R=15, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 6, 2, 3, 5])

In [ ]:
# check identified topics
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx+1, topic))

Topic: 1 
Words: 0.013*"station" + 0.012*"train" + 0.009*"train_station" + 0.007*"hous" + 0.007*"far" + 0.006*"beauti_apart" + 0.005*"veri_much" + 0.005*"car" + 0.004*"detail" + 0.004*"pick"
Topic: 2 
Words: 0.022*"transport" + 0.020*"public" + 0.019*"public_transport" + 0.012*"pleasant" + 0.010*"nice_place" + 0.007*"veri_nice_apart" + 0.007*"veri_nice_place" + 0.007*"host_veri" + 0.006*"pleasant_stay" + 0.006*"veri_pleasant"
Topic: 3 
Words: 0.010*"amaz_view" + 0.009*"love_apart" + 0.007*"locat_easi" + 0.007*"excel_locat" + 0.006*"appoint" + 0.005*"condit" + 0.005*"great_apart_great" + 0.005*"locat_super" + 0.005*"well_appoint" + 0.005*"tram_bu"
Topic: 4 
Words: 0.011*"apart" + 0.008*"stay" + 0.008*"nice" + 0.008*"recommend" + 0.007*"host" + 0.007*"perfect" + 0.006*"love" + 0.005*"view" + 0.005*"help" + 0.005*"walk"
Topic: 5 
Words: 0.016*"beauti_view" + 0.014*"opera" + 0.013*"opera_hous" + 0.012*"super_nice" + 0.011*"thank_veri" + 0.011*"calm" + 0.011*"thank_veri_much" + 0.010*"nice_

### Running LDA using Bag-of-Words without tf-idf scores

Here, LDA is conducted with the bow_corpus and without tf-idf representation.

In [ ]:
# initiate model
lda_model = gensim.models.ldamodel.LdaModel(corpus=bow_corpus, 
                                            id2word=id2word,
                                            num_topics=6,
                                            random_state=42,
                                            passes=5,
                                            alpha='auto')

### Visualizing topics

In [ ]:
# for visualization
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, id2word, mds='mmds', R=15)

/opt/conda/lib/python3.10/site-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


In [ ]:
# visualize
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.100981  0.265812       1        1  27.642929
3     -0.174947 -0.336627       2        1  20.931850
0     -0.370297 -0.067517       3        1  19.638660
2     -0.269347  0.360529       4        1  17.310946
1      0.255629 -0.337388       5        1  10.883627
4      0.457981  0.115192       6        1   3.591988, topic_info=               Term           Freq          Total Category  logprob  loglift
180           apart  290053.000000  290053.000000  Default  15.0000  15.0000
13            great  334749.000000  334749.000000  Default  14.0000  14.0000
46            place  248983.000000  248983.000000  Default  13.0000  13.0000
37             stay  356391.000000  356391.000000  Default  12.0000  12.0000
51        recommend  191112.000000  191112.000000  Default  11.0000  11.0000
...             ...            ...            ...      ...      ...      ...
1102     thank_much    5922.572603    5923.408900   Topic6  -4.5502   3.3263
1327          money    5888.830454    5889.668802   Topic6  -4.5559   3.3263
998   veri_pleasant    4491.160987    4491.997330   Topic6  -4.8269   3.3263
73             much   36807.266656   38110.955690   Topic6  -2.7233   3.2917
170         definit    9921.217246   56170.988848   Topic6  -4.0343   1.5927

[157 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
92        2  0.000542         access
92        4  0.999421         access
190       1  0.000149           also
190       2  0.332047           also
190       3  0.301449           also
...     ...       ...            ...
60        1  0.003960         wonder
60        3  0.297229         wonder
60        5  0.698804         wonder
718       2  0.999984           work
178       6  0.999949  would_definit

[237 rows x 3 columns], R=15, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 4, 1, 3, 2, 5])

In [ ]:
# check identified topics
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx+1, topic))

Topic: 1 
Words: 0.027*"time" + 0.023*"make" + 0.020*"hous" + 0.018*"home" + 0.017*"stay" + 0.016*"go" + 0.014*"thi" + 0.014*"even" + 0.014*"visit" + 0.013*"give"
Topic: 2 
Words: 0.171*"apart" + 0.061*"perfect" + 0.025*"love" + 0.025*"back" + 0.025*"view" + 0.025*"come" + 0.024*"wonder" + 0.016*"thi" + 0.014*"come_back" + 0.014*"question"
Topic: 3 
Words: 0.044*"walk" + 0.033*"close" + 0.023*"central" + 0.023*"easi" + 0.021*"restaur" + 0.020*"station" + 0.016*"area" + 0.014*"quiet" + 0.013*"lot" + 0.012*"train"
Topic: 4 
Words: 0.022*"room" + 0.015*"bed" + 0.014*"night" + 0.012*"kitchen" + 0.010*"space" + 0.010*"use" + 0.010*"small" + 0.010*"onli" + 0.010*"bathroom" + 0.009*"live"
Topic: 5 
Words: 0.103*"enjoy" + 0.066*"much" + 0.045*"enjoy_stay" + 0.029*"anyon" + 0.029*"definit_recommend" + 0.021*"realli_enjoy" + 0.021*"pleasant" + 0.018*"veri_much" + 0.018*"definit" + 0.016*"would_definit"
Topic: 6 
Words: 0.078*"great" + 0.070*"stay" + 0.058*"place" + 0.044*"recommend" + 0.041*"hos